In [1]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np
import pandas as pd
import pickle 

### helper functions

In [93]:
def helper_regex(text):
    match = re.search(rf"{'Full'}\s+(\w+)", text)
    if match:
        return match.group(1)
    return None

def cast_func(scan_number):
    scan_masses = []
    scan_intensities = []
    data_masses = []
    data_intensities = [] 


    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number)
    precursor = float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1])
    scan_masses = raw_scan.preferred_masses
    scan_intensities = raw_scan.preferred_intensities

    for i in range(4000, 20000): 
        data_intensities.append(0) 
        data_masses.append(i/10)

    for j in range(0,len(scan_masses)):
        for i in range(0,len(data_masses)):
            if (i+ 4000 + 1)/10 > scan_masses[j] > (i + 4000)/10: 
                data_intensities[i] = data_intensities[i] + scan_intensities[j]

    for i in range(0,16000):
        if (precursor -5) < data_masses[i] < (precursor +5): data_intensities[i] = 0
        
    return(list(data_intensities))

def cast_func_1amu(scan_number):
    scan_masses = []
    scan_intensities = []
    data_masses = []
    data_intensities = [] 
 


    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number)
    precursor = float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1])
  
    scan_masses = raw_scan.preferred_masses
    scan_intensities = raw_scan.preferred_intensities
   
    # for k in range(0, len(scan_intensities)):
    #     if scan_intensities[k] > (5/100)*max(scan_intensities): 
    #         scan_intensities1.append(scan_intensities[k])
    #         scan_masses1.append(scan_masses[k])


    for i in range(400, 2000): 
        data_intensities.append(0) 
        data_masses.append(i)

    for j in range(0,len(scan_masses)):
        for i in range(0,len(data_masses)):
            if (i+ 400 + 1) > scan_masses[j] > (i + 400): 
                data_intensities[i] = data_intensities[i] + scan_intensities[j]

    # for i in range(0,len(data_masses)):
    #     if (precursor -5) < data_masses[i] < (precursor +5): data_intensities[i] = 0
        
    return(list(data_intensities))



### select a raw file

In [13]:
# raw = RawFile("C:/20231215_msz1718_MAbPac-06_Neo_ACN1_TreatmentA_biorep01_techrep01.raw")
# raw = RawFile("C:/20231221_msz1718_MAbPac-02_Neo_ExtraValveT_ACN30S1_TreatmentA_biorep01_techrep01.raw")
raw = RawFile("C:/20240708_chm134_Nadig_MAbPac-04A_Proteo06_Group01_Time00_biorep01_techrep01.raw")

### generate a cast spectra database including scan index, retention time, precursor m/z values, and TIC

In [ ]:
cast_spectra = []
mz_value = []
tic = []
scan_number = []
retention_time = []

for i in range(1, raw.number_of_scans):
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
    if str(helper_regex(raw_scan.scan_type)) == 'ms2':
        cast_spectra.append(cast_func_1amu(i))
        mz_value.append(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]))
        tic.append(raw_scan.scan_statistics.tic)
        scan_number.append(raw_scan.scan_statistics.scan_number)
        retention_time.append(raw.get_retention_time_from_scan_number(raw_scan.scan_statistics.scan_number))
        print("scan", i, "is complete")

scan_dict = {'scan': scan_number, 'retntion time': retention_time, 'm/z': mz_value, 'TIC': tic, 'cast spectra': cast_spectra} 

with open('scan_dict_run1', 'wb') as f: 
     pickle.dump(scan_dict, f) 

print("completed")

In [ ]:
pd.DataFrame(scan_dict)

In [3]:
with open('scan_dict_run1', 'rb') as f: 
     scan_dict_run1 = pickle.load(f) 

# with open('scan_dict_run2', 'rb') as f: 
#      scan_dict_run2 = pickle.load(f) 

### search a complete run against a database

In [ ]:
pd.DataFrame(scan_dict_run1)

In [ ]:
database_1 = scan_dict_run1["cast spectra"]
database_2 = scan_dict_run2["cast spectra"]

mz_value_1 = scan_dict_run1["m/z"]
mz_value_2 = scan_dict_run2["m/z"]

scan_index_1 = scan_dict_run1["scan"]
scan_index_2 = scan_dict_run2["scan"]

similarity = []
match_scan = []

for i in range(0, len(database_2)):
    similarity.append(0)
for i in range(0, len(database_2)):
    match_scan.append(0)


for i in range(0, len(database_2)):
    for j in range(0, len(database_1)):
        if mz_value_2[i] - 1 < mz_value_1[j] < mz_value_2[i] + 1:
            X = [database_2[i], database_1[j]]
            if round(cosine_similarity(X)[1][0], 2) > similarity[i]: 
                similarity[i] = round(cosine_similarity(X)[1][0], 2)
                match_scan[i] = scan_index_1[j]
    print("the max similarity for index", i, "is", similarity[i], "found in index ", match_scan[i])

In [102]:
scan_dict_run2['similarity'] = similarity
scan_dict_run2['match_scan'] = match_scan
del scan_dict_run2['cast spectra']

In [105]:
df = pd.DataFrame(scan_dict_run2)
df.to_csv('result.csv')


### run a single spectra against a database

In [ ]:
ref_scan = cast_func_1amu(4291)
scan_id = []
scan_sim = []
precursor = []



for i in range(1, raw.number_of_scans):
    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=i)
    if str(helper_regex(raw_scan.scan_type)) == 'ms2':
        X = [ref_scan, cast_func_1amu(i)]
        result = round(cosine_similarity(X)[1][0], 2)       
        precursor.append(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]))
        scan_id.append(i)
        scan_sim.append(result)
        print(float(re.findall(r'[\d]*[.][\d]+', raw_scan.scan_type)[1]), "in scan", i, "is", result)



In [120]:
result_match = {'scan_id': scan_id, 'scan_sim': scan_sim, 'precursor': precursor}

In [122]:
a = pd.DataFrame(result_match)
a.to_csv("result_match.csv")

In [ ]:
pd.DataFrame(scan_dict_run1)

In [81]:
X = [cast_func(1000), cast_func(1000)]
result = round(cosine_similarity(X)[1][0], 2)  
print(result)

1.0


In [102]:
for i in [1010]:
    X = [cast_func_1amu(1009), cast_func_1amu(i)]
    result = round(cosine_similarity(X)[1][0], 2)  
    print(i, result)

1010 0.0
